In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2OXGBoostEstimator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

h2o.init()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## csv File reading

In [ ]:
train_df = h2o.import_file("/kaggle/input/tabular-playground-series-dec-2021/train.csv")
test_df = h2o.import_file("/kaggle/input/tabular-playground-series-dec-2021/test.csv")

In [ ]:
train_df.describe()

Some simple data rangement.

In [ ]:
x = [i for i in train_df.columns if i not in {"Id", "Cover_Type"}]
y = "Cover_Type"

train_df[y] = train_df[y].asfactor()

## Data Training

In [ ]:
# ## automl
# aml = H2OAutoML(max_models=10, seed=1234, verbosity="info", max_runtime_secs=60 * 20)
# aml.train(x=x, y=y, training_frame=train_df)

## XGBoost model
train, valid = train_df.split_frame(ratios=[.8], seed=1234)
xgb = H2OXGBoostEstimator(booster='dart', seed=1234, backend="gpu")
xgb.train(x=x, y=y, training_frame=train, validation_frame=valid)

In [ ]:
# ## automl leaderboard
# lb = aml.leaderboard
# lb.head(rows=lb.nrows)

# xgboost performance
xgb.model_performance()

In [ ]:
# xgb.explain(train_df)

In [ ]:
# preds = aml.predict(test_df)
preds = xgb.predict(test_df)

In [ ]:
df = preds.as_data_frame()
df['Id'] = test_df.as_data_frame()['Id']
res_df = df[['Id', 'predict']]
res_df.columns = ['Id', 'Cover_Type']

In [ ]:
res_df.to_csv('/kaggle/working/submission.csv', index=False)